In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages = [HumanMessage(content="What is the weather in Yerevan?")]

In [9]:
thread = {"configurable": {"thread_id": "1"}}

In [10]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr', 'function': {'arguments': '{\n  "query": "current weather in Yerevan"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 153, 'total_tokens': 178}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5e6c71d4a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5afa8181-6f4e-4422-a9b8-f1c26b720932-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Yerevan'}, 'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr'}], usage_metadata={'input_tokens': 153, 'output_tokens': 25, 'total_tokens': 178})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Yerevan'}, 'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr'}
Back to the model!
[ToolMessage(content="[{'url': 'https://www.accuweather.com/en/am/yerevan/16890/weather-forecast/16890', 'conten

In [11]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Yerevan?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr', 'function': {'arguments': '{\n  "query": "current weather in Yerevan"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 153, 'total_tokens': 178}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5e6c71d4a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5afa8181-6f4e-4422-a9b8-f1c26b720932-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Yerevan'}, 'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr'}], usage_metadata={'input_tokens': 153, 'output_tokens': 25, 'total_tokens': 178}), ToolMessage(content="[{'url': 'https://www.accuweather.com/en/am/yerevan/16890/weather-forecast/16890', 'content': 'Yerevan, Yerevan, Armenia Weather Forecast, with current con

In [12]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Cr6sD1Hko8QCxEiCw7qi1z1u', 'function': {'arguments': '{"query": "current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_yA8RfXnMYMw97qDUaiubC0bM', 'function': {'arguments': '{"query": "current weather in LA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 436, 'total_tokens': 495}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e5ce3555-5940-4a4b-b656-f360c3e8ad74-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_Cr6sD1Hko8QCxEiCw7qi1z1u'}, {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA'}, 'id': 'call_yA8RfXnMYMw97qDUaiubC0bM'}], usage_metadata={'input_tokens': 436, 'output_t

In [13]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Yerevan?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr', 'function': {'arguments': '{\n  "query": "current weather in Yerevan"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 153, 'total_tokens': 178}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5e6c71d4a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5afa8181-6f4e-4422-a9b8-f1c26b720932-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Yerevan'}, 'id': 'call_LOhP3JiEZXihrg5YmAzPDPZr'}], usage_metadata={'input_tokens': 153, 'output_tokens': 25, 'total_tokens': 178}), ToolMessage(content="[{'url': 'https://www.accuweather.com/en/am/yerevan/16890/weather-forecast/16890', 'content': 'Yerevan, Yerevan, Armenia Weather Forecast, with current con

In [14]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="To determine which city is currently warmer, let's check the specific temperature details from the provided sources:\n\n1. **Yerevan, Armenia:**\n   - You would need to check the current temperature from one of the provided weather websites for Yerevan, such as [AccuWeather - Current Weather in Yerevan](https://www.accuweather.com/en/am/yerevan/16890/current-weather/16890).\n\n2. **Los Angeles, California:**\n   - According to the [WeatherAPI](https://www.weatherapi.com/) result, the current temperature in Los Angeles is approximately 22.8°C (73°F).\n\nPlease check the current temperature for Yerevan from the linked source to compare it with the 22.8°C in Los Angeles. If you want, I can look it up directly for you.", response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 1617, 'total_tokens': 1788}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'stop', 'logprobs': None}, id='run-599800a

In [15]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please specify the items or locations you are comparing for warmth? For instance, are you comparing two geographical locations, two types of clothing, seasons, etc.?', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 149, 'total_tokens': 183}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_9cb5d38cf7', 'finish_reason': 'stop', 'logprobs': None}, id='run-59659e53-cab1-42a8-a1df-e4b4993b4683-0', usage_metadata={'input_tokens': 149, 'output_tokens': 34, 'total_tokens': 183})]}


In [8]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [9]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

/Users/kjavadyan/projects/jupyterspace/venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_G6H8NWIGJCLFt9BDEz1kS3ue'}
Back to the model!
The| current| weather| in| San| Francisco| is| partly| cloudy| with| a| temperature| of| |16|.|1|°C| (|61|.|0|°F|).| The| wind| is| blowing| from| the| southwest| at| |3|.|6| k|ph| (|2|.|2| mph|).| The| humidity| is| at| |70|%,| and| visibility| is| around| |16| kilometers| (|9| miles|).| The| UV| index| is| |5|.|